# Jupyter notebook used to calculate viscosity from the Giordano et al. (2008) model (https://doi.org/10.1016/j.epsl.2008.03.038). 
### Input excel spreadsheet must have oxides in order and formated as in the code block 3 blocks below (i.e., sio2, tio2,....). Spreadsheet must also have a column titled T_C that has estimated temperatures in °C.
### Input file path to the excel spreadsheet in the parentheses labelled 'path' below. Same with the sheet name.
### Go to Run -> Run all cells. An excel spreadhsheet (specifically, .xlsx) titled 'viscosity_df' will be exported to the folder that the jupyter notebook resides in.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = '/Users/ctlewis/Documents/Projects/Newberry maybe/Data/Whole_rx_KW_CTL/wsudata_062022.xlsx'
sheet = 'viscosity'

data = pd.read_excel(path,
                     sheet_name=sheet)
majors = data.loc[:,'sio2':'f2o']
samples = data.loc[:,'sample']
majors['sample'] = samples

In [3]:
majors

,sio2,tio2,al2o3,feo,mno,mgo,cao,na2o,k2o,p2o5,h2o,co2,f2o,sample
0,74.133695,0.194283,14.275861,1.734563,0.043996,0.149361,0.907063,4.135443,4.399317,0.026418,3,0,0,NB21-02
1,71.625532,0.291031,14.756922,2.707745,0.086276,0.216186,1.139575,6.105298,3.019351,0.052083,3,0,0,NB21-04
2,69.388022,0.452462,15.110310,3.972503,0.132600,0.429912,1.828243,5.928982,2.665205,0.091760,3,0,0,NB21-05
3,73.090630,0.290178,13.353448,2.827193,0.086760,0.341898,1.717885,4.603617,3.358137,0.330254,3,0,0,NB21-06B
4,65.890776,0.851369,15.500732,4.617694,0.111308,1.615259,3.718150,4.845881,2.545235,0.303596,3,0,0,NB21-07
5,55.147480,1.801742,16.002833,9.853010,0.177487,4.289087,7.436680,3.805619,1.062599,0.423464,3,0,0,NB21-11B
6,68.506457,0.530054,17.820949,3.953569,0.096588,0.469097,1.165927,4.039425,3.344420,0.073514,3,0,0,NB21-12
7,69.729936,0.493702,14.579447,4.367965,0.125757,0.501037,1.923770,5.248894,2.938002,0.091488,3,0,0,NB21-11A-L
8,70.692092,0.357957,14.527275,3.708390,0.117127,0.317140,1.586118,5.526778,3.105322,0.061800,3,0,0,NB21-11C
9,72.747621,0.253306,14.414509,2.249053,0.065297,0.207558,0.910777,5.075030,4.025472,0.051378,3,0,0,NB21-14


In [4]:
oxide_dict = {'oxide': ['sio2','tio2','al2o3','feo','mno','mgo','cao','na2o','k2o','p2o5','h2o','co2','f2o'],
              'Molecular Mass (g/mol)': [60.08,79.886,101.96,71.844,70.9374,40.3044,56.0774,61.9789,94.2,283.889,18.02,44.01,53.9962]
             }

In [5]:
# define VFT constants
A = -4.55

b1 = 159.6
b2 = -173.3
b3 = 72.1
b4 = 75.7
b5 = -39.0
b6 = -84.1
b7 = 141.5
b11 = -2.43
b12 = -0.91
b13 = 17.6

c1 = 2.75
c2 = 15.7
c3 = 8.3
c4 = 10.2
c5 = -12.3
c6 = -99.5
c11 = 0.30

# put the b coefficients in a list to be looped through
b_ = [b1, b2, b3, b4, b5, b6, b7, b11, b12, b13] 
c_ = [c1, c2, c3, c4, c5, c6, c11]

In [10]:
def get_mol_percent(weights):
    """
    function to get mol% of the input dataframe. Called in the visocsity function below.
    
    Inputs:
    'weights': list of values corresponding to wt% of 'sio2','tio2','al2o3','feo','mno','mgo','cao','na2o','k2o','p2o5','h2o','co2','f2o' for a rock analysis.
    
    Returns:
    oxide_df: a df that is used to calculate viscosities in the function below.
    majors: the global variable of just oxide data set in the second code block above.
    """
    
    oxide_dict = {'oxide': ['sio2','tio2','al2o3','feo','mno','mgo','cao','na2o','k2o','p2o5','h2o','co2','f2o'],
              'Molecular Mass (g/mol)': [60.08,79.88,101.96,71.85,70.94,40.3,56.08,61.99,94.2,283.889,18.02,44.01,53.9962]
             }
    oxide_dict['wt%'] = weights # put wt% of single sample from dataframe into dictionary
    oxide_df = pd.DataFrame.from_dict(oxide_dict) # put dictionary in a df
    oxide_df['mol'] = oxide_df['wt%'] / oxide_df['Molecular Mass (g/mol)'] # calculate moles from wt %
    oxide_df['mol%'] = oxide_df['mol'] / oxide_df['mol'].sum() * 100 # calculate mol %
    oxide_df = oxide_df.set_index('oxide')
    oxide_df = oxide_df.T
    
    return oxide_df


def get_visc(df,majors):
    """
    Function that takes the input data and calculates temperature dependent viscosities following the model of Giordano et al. (2008). 
    
    Inputs: 
    df: The input dataframe from the spreadsheet with columns titled as prescribed above.
    majors: The global variable
    
    """
    # set up empty arrays to be filled
    log_visc = []
    visc = []
    
    # run a for loop across the length of the samples
    for i in range(0,len(majors)):
        tosend_wt = list(majors.iloc[i,:-1].values) # get wt%'s from the ith sample
        data_mol = get_mol_percent(tosend_wt) # get a local df that has mol%'s
        sample_index = list(majors['sample'].values)[i] # get a sample list
        
        T = list(data['T_C'].values)[i] + 273 # put temp in K
        
        #calculate all the individual parameters that are functions of constant and mol %'s
        b1_ = b_[0] * (data_mol.sio2['mol%'] + data_mol.tio2['mol%'])
        b2_ = b_[1] * (data_mol.al2o3['mol%'])
        b3_ = b_[2] * (data_mol.feo['mol%'] + data_mol.mno['mol%'] + data_mol.p2o5['mol%'])
        b4_ = b_[3] * (data_mol.mgo['mol%'])
        b5_ = b_[4] * (data_mol.cao['mol%'])
        b6_ = b_[5] * (data_mol.na2o['mol%'] + (data_mol.h2o['mol%'] + data_mol.f2o['mol%']))
        b7_ = b_[6] * ((data_mol.h2o['mol%'] + data_mol.f2o['mol%']) + np.log(1+ data_mol.h2o['mol%']))
        b11_ = b_[7] * ((data_mol.sio2['mol%'] + data_mol.tio2['mol%']) * (data_mol.feo['mol%'] + data_mol.mgo['mol%'] + data_mol.mno['mol%']))
        b12_ = b_[8] * ((data_mol.sio2['mol%'] + (data_mol.tio2['mol%'] + data_mol.al2o3['mol%']) + data_mol.p2o5['mol%']) * 
                         ((data_mol.na2o['mol%'] + data_mol.k2o['mol%']) + data_mol.h2o['mol%']))
        b13_ = b_[9] * (data_mol.al2o3['mol%'] * (data_mol.na2o['mol%'] + data_mol.k2o['mol%']))
        
        c1_ = c_[0] * data_mol.sio2['mol%']
        c2_ = c_[1] * (data_mol.tio2['mol%'] + data_mol.al2o3['mol%'])
        c3_ = c_[2] * (data_mol.feo['mol%'] + data_mol.mgo['mol%'] + data_mol.mno['mol%'])
        c4_ = c_[3] * data_mol.cao['mol%']
        c5_ = c_[4] * (data_mol.na2o['mol%'] + data_mol.k2o['mol%'])
        c6_ = c_[5] * np.log(1 + (data_mol.h2o['mol%'] + data_mol.f2o['mol%']))
        c11_ = c_[6] * ((data_mol.al2o3['mol%'] + (data_mol.feo['mol%']+data_mol.mgo['mol%']+data_mol.mno['mol%']) + data_mol.cao['mol%'] - data_mol.p2o5['mol%']) * 
                        ((data_mol.na2o['mol%']+data_mol.k2o['mol%']) + (data_mol.h2o['mol%']+data_mol.f2o['mol%'])))
        
        # sum appropriately. Eqn's 2 & 3 in Giordano et al.
        B = (b1_+b2_+b3_+b4_+b5_+b6_+b7_) + (b11_+b12_+b13_)
        C = (c1_+c2_+c3_+c4_+c5_+c6_) + c11_
        
        # VFT equation
        log_eta = A + B/(T-C)
        eta = np.exp(log_eta)
        
        # append data
        log_visc.append(log_eta)
        visc.append(eta)
        
        # put b and c in lists
        blist = [b1_, b2_, b3_, b4_, b5_, b6_, b7_, b11_, b12_, b13_]
        clist = [c1_, c2_, c3_, c4_, c5_, c6_, c11_]
    
    # set up output data
    output_df = df
    output_df['log eta'] = log_visc
    output_df['eta'] = visc
        
    return output_df,B,C,blist,clist
        
        

In [11]:
visc_df,B,C,bs,cs = get_visc(data,majors)

In [12]:
visc_df

,sample,unit,age (ka),sio2,tio2,al2o3,feo,mno,mgo,cao,na2o,k2o,p2o5,h2o,co2,f2o,T_C,log eta,eta
0,NB21-02,McKay Butte,600.0,74.133695,0.194283,14.275861,1.734563,0.043996,0.149361,0.907063,4.135443,4.399317,0.026418,3,0,0,750,6.467998,644.192443
1,NB21-04,Paulina Peak,80.0,71.625532,0.291031,14.756922,2.707745,0.086276,0.216186,1.139575,6.105298,3.019351,0.052083,3,0,0,750,5.969058,391.136981
2,NB21-05,Evans Well,100.0,69.388022,0.452462,15.110310,3.972503,0.132600,0.429912,1.828243,5.928982,2.665205,0.091760,3,0,0,750,5.850797,347.511199
3,NB21-06B,Tepee Draw,300.0,73.090630,0.290178,13.353448,2.827193,0.086760,0.341898,1.717885,4.603617,3.358137,0.330254,3,0,0,750,6.296173,542.491871
4,NB21-07,Qdt,220.0,65.890776,0.851369,15.500732,4.617694,0.111308,1.615259,3.718150,4.845881,2.545235,0.303596,3,0,0,750,5.804576,331.814442
5,NB21-11B,Pumice Flat,150.0,55.147480,1.801742,16.002833,9.853010,0.177487,4.289087,7.436680,3.805619,1.062599,0.423464,3,0,0,750,5.083114,161.275486
6,NB21-12,Ice Quarry,75.0,68.506457,0.530054,17.820949,3.953569,0.096588,0.469097,1.165927,4.039425,3.344420,0.073514,3,0,0,750,6.208477,496.943598
7,NB21-11A-L,Pumice Flat,150.0,69.729936,0.493702,14.579447,4.367965,0.125757,0.501037,1.923770,5.248894,2.938002,0.091488,3,0,0,750,5.952938,384.882511
8,NB21-11C,Pumice Flat,150.0,70.692092,0.357957,14.527275,3.708390,0.117127,0.317140,1.586118,5.526778,3.105322,0.061800,3,0,0,750,5.968396,390.878350
9,NB21-14,PCT,55.0,72.747621,0.253306,14.414509,2.249053,0.065297,0.207558,0.910777,5.075030,4.025472,0.051378,3,0,0,750,6.182214,484.062565


In [13]:
visc_df.to_excel('viscosity_df.xlsx')